In [1]:
import pandas as pd
# 과학적 표기 안하도록
pd.options.display.float_format = '{:.2f}'.format

import numpy as np
import datetime

---
### request+bs로 크롤링


In [2]:
import requests
from bs4 import BeautifulSoup as bs

In [3]:
# code로 접근 가능한 url
code_url = "https://finance.naver.com/item/main.nhn?code="
# 종목 분석 url
coinfo_url = "https://finance.naver.com/item/coinfo.nhn?code="
# 종목 분석 url (!naver)
coinfo_url_2 = "https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cmp_cd=%s&cn="
# 시가총액 목록 url
size_market_url_0 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="
size_market_url_1 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page="

---

### 시가총액 목록 사용하기

In [4]:
# 시가총액 300억 이하 목록 구하기
codeL = []

# 코스피 300억 이하 (후보)
for i in range(19, 33):
    soup = bs(requests.get(size_market_url_0+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL

# 코스닥 300억 이하 (후보)
for i in range(22, 30):
    soup = bs(requests.get(size_market_url_1+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL
    

In [5]:
# 오늘의 시가총액 code 목록 저장 
todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
f = open("./"+todayDate+"_시가총액_하위권.txt", 'w')
f.write("\n".join(codeL))
f.close()

# 오늘의 시가총액 code 목록 불러오기
useDate = todayDate
# useDate = "2020_Feb_18"
f = open("./"+useDate+"_시가총액_하위권.txt", "r")
codeL = [x.replace("\n", "") for x in f.readlines()]
f.close()

---

<b>base code</b>


In [6]:
def getSoupFromCode(code):
    code_url = "https://finance.naver.com/item/main.nhn?code="
    soup = bs(requests.get(code_url + code).text, 'html.parser')
    return soup

def cleanText(text):
    return text.replace("\n", "").replace("\t","")

def getName(code):
    soup = getSoupFromCode(code)
    name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
    return name

def getSoupFromReport(code):
    report_url = "https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd="
    soup = bs(requests.get(report_url + code).text, 'html.parser')
    return soup

---

## 사용할 조건들

- PBR 1 미만
- PER 7 미만
- 시총 300 미만
- PCR 8 미만
- POR 8 미만


---

#### etf 인가

In [7]:
def isETF(code):
    soup = getSoupFromCode(code)
    if(len(soup.select(".e_summary")) == 0):
        return False
    return True

#### etn 인가

In [8]:
def isETN(code):
    name = getName(code)
    if("ETN" in name):
        return True
    return False

#### 시총 300억 미만인가

In [9]:
def under300(code):
    soup = getSoupFromCode(code)
    
    if(int(cleanText(soup.select_one("#_market_sum").text)) < 300):
        return True
    return False    

#### per 7 미만인가

In [19]:
def perUnder7(code):
    soup = getSoupFromReport(code)
    try:
        temp = [x for x in soup.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PER"]
    except:
        return -1
    per_str = [x.text.split()[1] for x in soup.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PER"][0]
    if(float(per_str) < 7 and float(per_str) > 0):
        return per_str
    return -1
    

#### pbr 1 미만인가

In [20]:
def pbrUnder1(code):
    soup = getSoupFromReport(code)
    try:
        temp = [x for x in soup.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PBR"]
    except:
        return -1
    pbr_str = [x.text.split()[1] for x in soup.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PBR"][0]
    if(float(pbr_str) < 1 and float(pbr_str) > 0):
        return pbr_str
    return -1


In [21]:
good = []

for code in codeL:
    if(isETF(code)):
        continue
    if(isETN(code)):
        continue
    if(under300(code) == False):
        continue
    print(code)
    per = perUnder7(code)
    if(per == -1):
        continue
    pbr = pbrUnder1(code)
    if(pbr == -1):
        continue
    name = getName(code)
    good.append(name + " " + str(per) + " " + str(pbr))
    print(name, per, pbr)

005820
012170
004100
003780
진양산업 3.92 0.65
123700
SJM 2.98 0.16
000910
018470
172580
하이골드12호 3.68 0.76
103130
005305
33637L


IndexError: list index out of range

---

## 나의 네이버증권 셋팅하기

In [13]:
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys

import time

In [14]:
# 미리 로그인
driver = webdriver.Chrome('./chromedriver')
itemListUrl = "https://finance.naver.com/mystock/itemList.nhn?groupId=6&type=OVERALL"
driver.get(itemListUrl)

In [16]:
for group in range(6, 9):

    # 링크 접속 및 로그인
    itemListUrl = "https://finance.naver.com/mystock/itemList.nhn?groupId=%s&type=OVERALL" % str(group)
    print(itemListUrl )
    
    # 기존 관심 종목 삭제
    time.sleep(0.5)
    driver.get(itemListUrl)
    time.sleep(0.5)
    driver.find_element_by_css_selector("#all_select").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector("#wrap > div.section_mys > div.group_mystb.NE\=a\:lst > div:nth-child(2) > button.btn_del.NPI\=a\:del").click()
    time.sleep(0.5)
    Alert(driver).accept()
    
    # 적합 종목 리스트 찾기
    if(group == 6):
        useL = score25_code.copy()
    elif(group == 7):
        useL = score23_code.copy()
    elif(group == 8):
        useL = score20_code.copy()
            
    # 관심종목 목록 등록
    for nowCode in useL:
        time.sleep(0.5)
        driver.find_element_by_css_selector("#mystock_input").clear()
        time.sleep(0.5)
        driver.find_element_by_css_selector("#mystock_input").send_keys(nowCode)
        driver.find_element_by_css_selector("#mystock_input").send_keys(Keys.RETURN)


https://finance.naver.com/mystock/itemList.nhn?groupId=6&type=OVERALL
https://finance.naver.com/mystock/itemList.nhn?groupId=7&type=OVERALL
https://finance.naver.com/mystock/itemList.nhn?groupId=8&type=OVERALL
